<a href="https://colab.research.google.com/github/shivasaxena23/MSDNet-PyTorch/blob/master/Syft_CIFAR10_FairFed_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install syft==0.2.9

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# import syft as sy  # <-- NEW: import the Pysyft library


In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 16
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 200
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print(device)
criterion = nn.CrossEntropyLoss()


cuda


In [4]:
# hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# charlie = sy.VirtualWorker(hook, id="charlie")
# bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
# alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice
# compute_nodes = [alice, bob, charlie]
# secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [5]:
# def load_data():
    
#     '''<--Load CIFAR dataset from torch vision module distribute to workers using PySyft's Federated Data loader'''
    

#     federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.CIFAR10('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                    ]))
#     .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

#     test_loader = torch.utils.data.DataLoader(
#     datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
#     return federated_train_loader,test_loader

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# get some random training images
dataiter = iter(alice_train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(64)))

NameError: ignored

In [7]:
def load_data_non_iid():

    alice_subset = []
    bob_subset = []
    charlie_subset = []
    
    for i in range(50000):
      if i%5 == 0:
        alice_subset.append(i)
      elif i%3 == 0:
        bob_subset.append(i)
      elif i%2 == 0:
        charlie_subset.append(i)

        

    alice_train_loader = torch.utils.data.DataLoader( 
    datasets.CIFAR10('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(alice_subset), **kwargs)

    for i in range(50000):
      if i%2 == 0:
        bob_subset.append(i)

    bob_train_loader = torch.utils.data.DataLoader( 
    datasets.CIFAR10('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(bob_subset), **kwargs)
    
    charlie_train_loader = torch.utils.data.DataLoader( 
    datasets.CIFAR10('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(charlie_subset), **kwargs)
      
    #^ Change the batch sizes for 3 different

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return alice_train_loader, bob_train_loader, charlie_train_loader,test_loader

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# model1 = Net().to(device)
# model2 = Net().to(device)
# print(model1.fc3.weight.data[0][0])
# print(model2.fc3.weight.data[0][0])

In [10]:
# sdA = model1.state_dict()
# sdB = model2.state_dict()

# # Average all parameters
# for key in sdA:
#     sdB[key] = (sdB[key] + sdA[key])

# # Recreate model and load averaged state_dict (or use modelA/B)
# model = Net().to(device)
# print(model.fc3.weight.data[0][0])
# model.load_state_dict(sdB)
# print(model.fc3.weight.data[0][0])

In [11]:
# def train(args, model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
#         model.send(data.location) # <-- NEW: send the model to the right location
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()
#         model.get() # <-- NEW: get the model back
#         if batch_idx % args.log_interval == 0:
#             loss = loss.get() # <-- NEW: get the loss back
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))


In [17]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    model.to(device)
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [18]:
def subtract(model1, model2):

  params1 = model1.named_parameters()
  params2 = model2.named_parameters()

  dict_params2 = dict(params2)

  for name1, param1 in params1:
      if name1 in dict_params2:
          dict_params2[name1].data.copy_(param1.data - dict_params2[name1].data)
  
  model = Net().to(device)
  model.load_state_dict(dict_params2)

  return model

In [19]:
def combine(model1, model2, model3, alpha, beta, gamma):
  
  params1 = model1.named_parameters()
  params2 = model2.named_parameters()
  params3 = model3.named_parameters()
  
  dict_params2 = dict(params2)
  dict_params3 = dict(params3)

  for name1, param1 in params1:
      if name1 in dict_params2:
        if name1 in dict_params3:
          dict_params2[name1].data.copy_(alpha*param1.data + beta*dict_params2[name1].data + gamma*dict_params3[name1].data)
  
  model = Net().to(device)
  model.load_state_dict(dict_params2)
  
  return model

In [20]:
def train_non_iid(args, model_in, device, train_loader, optimizer_in, epoch, compute_id):
    #mod data based on the node mod 1, 2 and 3 size of data set gradient descent (not stochastic)
    grad = []
    model_old = copy.deepcopy(model_in)
    model_in.to(device)
    model_in.train()
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = data.to(device), target.to(device)
        optimizer_in.zero_grad()
        # print(model.conv1.weight.data[0][0][0][0],model_old.conv1.weight.data[0][0][0][0])
        output = model_in(data)
        loss = criterion(output, target)
        a = list(model_in.parameters())[0].clone()
        loss.backward()
        optimizer_in.step()
        b = list(model_in.parameters())[0].clone()
        print(torch.equal(a.data, b.data))
        if batch_idx % args.log_interval == 0:
            # loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    

    return subtract(model_in,model_old)

In [21]:
import copy


In [23]:
#Different size of dataset, same accuracy 

#<--Load federated training data and test data
#federated_train_loader,test_loader=load_data()

##Modified

alice_train_loader,bob_train_loader,charlie_train_loader,test_loader=load_data_non_iid()

#<--Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1) #<--TODO momentum is not supported at the moment

   
alices_model = copy.deepcopy(model)
bobs_model = copy.deepcopy(model)
charlies_model = copy.deepcopy(model)

alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.01, momentum=0.9)
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.01, momentum=0.9)
charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.01, momentum=0.9)

alices_model_prev = copy.deepcopy(model)
bobs_model_prev = copy.deepcopy(model)
charlies_model_prev = copy.deepcopy(model)

#<--Train Neural network and validate with test set after completion of training every epoch


for epoch in range(1, args.epochs+1):
    

    model_a = train_non_iid(args, alices_model, device, alice_train_loader, alices_opt, epoch,0)
    model_b = train_non_iid(args, bobs_model, device, bob_train_loader, bobs_opt, epoch,1)
    model_c = train_non_iid(args, charlies_model, device, charlie_train_loader, charlies_opt, epoch,2)


    combined_model = combine(model_a,model_b,model_c,2/11,3/11,6/11)
    #combined_model = combine(model_a,model_b,model_c,1/3,1/3,1/3)

    parmod_a = copy.deepcopy(alices_model_prev)
    parmod_b = copy.deepcopy(bobs_model_prev)
    parmod_c = copy.deepcopy(charlies_model_prev)

    # alices_model = combine(parmod_a,model_a,combined_model,1,2/3,1/3)
    # bobs_model = combine(parmod_b,model_b,combined_model,1,1/2,1/2)
    # charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

    alices_model = combine(parmod_a,model_a,combined_model,1,0,1)
    bobs_model = combine(parmod_b,model_b,combined_model,1,0,1)
    charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.01, momentum=0.9)
    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.01, momentum=0.9)
    charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.01, momentum=0.9)

    alices_model_prev = copy.deepcopy(alices_model)
    bobs_model_prev = copy.deepcopy(bobs_model)
    charlies_model_prev = copy.deepcopy(charlies_model)
    
    test(args, charlies_model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

Test set: Average loss: 0.0020, Accuracy: 2790/10000 (28%)


Test set: Average loss: 0.0016, Accuracy: 4345/10000 (43%)


Test set: Average loss: 0.0014, Accuracy: 4913/10000 (49%)


Test set: Average loss: 0.0013, Accuracy: 5306/10000 (53%)


Test set: Average loss: 0.0013, Accuracy: 5409/10000 (54%)


Test set: Average loss: 0.0012, Accuracy: 5668/10000 (57%)


Test set: Average loss: 0.0012, Accuracy: 5811/10000 (58%)


Test set: Average loss: 0.0012, Accuracy: 5884/10000 (59%)


Test set: Average loss: 0.0011, Accuracy: 5963/10000 (60%)


Test set: Average loss: 0.0011, Accuracy: 6033/10000 (60%)


Test set: Average loss: 0.0011, Accuracy: 6013/10000 (60%)


Test set: Average loss: 0.0011, Accuracy: 6034/10000 (60%)


Test set: Average loss: 0.0011, Accuracy: 6073/10000 (61%)


Test set: Average loss: 0.0012, Accuracy: 6027/10000 (60%)


Test set: Average loss: 0.0012,

In [24]:
    test(args, alices_model, device, test_loader)
    test(args, bobs_model, device, test_loader)
    test(args, charlies_model, device, test_loader)


Test set: Average loss: 0.0012, Accuracy: 6033/10000 (60%)


Test set: Average loss: 0.0012, Accuracy: 6033/10000 (60%)


Test set: Average loss: 0.0012, Accuracy: 6033/10000 (60%)



In [26]:
#Different size of dataset, different accuracy 

#<--Load federated training data and test data
#federated_train_loader,test_loader=load_data()

##Modified

alice_train_loader,bob_train_loader,charlie_train_loader,test_loader=load_data_non_iid()

# remote_dataset = (list(), list(), list())
# train_distributed_dataset = []

# for batch_idx, (data,target) in enumerate(alice_train_loader):
#       data = data.send(alice)
#       target = target.send(alice)
#       remote_dataset[0].append((data, target))
# for batch_idx, (data,target) in enumerate(bob_train_loader):
#       data = data.send(bob)
#       target = target.send(bob)
#       remote_dataset[1].append((data, target))
# for batch_idx, (data,target) in enumerate(charlie_train_loader):
#       data = data.send(charlie)
#       target = target.send(charlie)
#       remote_dataset[2].append((data, target))

#<--Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1) #<--TODO momentum is not supported at the moment

   
alices_model = copy.deepcopy(model)
bobs_model = copy.deepcopy(model)
charlies_model = copy.deepcopy(model)

alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.01, momentum=0.9)
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.01, momentum=0.9)
charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.01, momentum=0.9)

alices_model_prev = copy.deepcopy(model)
bobs_model_prev = copy.deepcopy(model)
charlies_model_prev = copy.deepcopy(model)

#<--Train Neural network and validate with test set after completion of training every epoch


for epoch in range(1, 30):
    

    model_a = train_non_iid(args, alices_model, device, alice_train_loader, alices_opt, epoch,0)
    model_b = train_non_iid(args, bobs_model, device, bob_train_loader, bobs_opt, epoch,1)
    model_c = train_non_iid(args, charlies_model, device, charlie_train_loader, charlies_opt, epoch,2)


    combined_model = combine(model_a,model_b,model_c,6/31,10/31,15/31)
    
    parmod_a = copy.deepcopy(alices_model_prev)
    parmod_b = copy.deepcopy(bobs_model_prev)
    parmod_c = copy.deepcopy(charlies_model_prev)

    # for key in sda:
    #   sda[key] = sda[key] + (2*grad_a[key] + grad_combine[key])/3

    # for key in sdb:
    #   sdb[key] = sdb[key] + (grad_b[key] + grad_combine[key])/2
      
    # for key in sdc:
    #   sdc[key] = sdc[key] + grad_combine[key]

    alices_model = combine(parmod_a,model_a,combined_model,1,3/5,2/5)
    bobs_model = combine(parmod_b,model_b,combined_model,1,2/5,3/5)
    charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001, momentum=0.9)
    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001, momentum=0.9)
    charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.001, momentum=0.9)

    alices_model_prev = copy.deepcopy(alices_model)
    bobs_model_prev = copy.deepcopy(bobs_model)
    charlies_model_prev = copy.deepcopy(charlies_model)
    
    test(args, charlies_model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

Test set: Average loss: 0.0019, Accuracy: 3269/10000 (33%)


Test set: Average loss: 0.0017, Accuracy: 3982/10000 (40%)


Test set: Average loss: 0.0016, Accuracy: 4345/10000 (43%)


Test set: Average loss: 0.0015, Accuracy: 4495/10000 (45%)


Test set: Average loss: 0.0015, Accuracy: 4627/10000 (46%)


Test set: Average loss: 0.0014, Accuracy: 4761/10000 (48%)


Test set: Average loss: 0.0014, Accuracy: 4842/10000 (48%)


Test set: Average loss: 0.0014, Accuracy: 4897/10000 (49%)


Test set: Average loss: 0.0014, Accuracy: 4970/10000 (50%)


Test set: Average loss: 0.0014, Accuracy: 5040/10000 (50%)


Test set: Average loss: 0.0014, Accuracy: 5087/10000 (51%)


Test set: Average loss: 0.0013, Accuracy: 5169/10000 (52%)


Test set: Average loss: 0.0013, Accuracy: 5267/10000 (53%)


Test set: Average loss: 0.0013, Accuracy: 5277/10000 (53%)


Test set: Average loss: 0.0013,

In [27]:
    test(args, alices_model, device, test_loader)
    test(args, bobs_model, device, test_loader)
    test(args, charlies_model, device, test_loader)


Test set: Average loss: 0.0013, Accuracy: 5479/10000 (55%)


Test set: Average loss: 0.0012, Accuracy: 5909/10000 (59%)


Test set: Average loss: 0.0012, Accuracy: 5909/10000 (59%)



In [28]:
#Different size of dataset, same accuracy 

#<--Load federated training data and test data
#federated_train_loader,test_loader=load_data()

##Modified

alice_train_loader,bob_train_loader,charlie_train_loader,test_loader=load_data_non_iid()

#<--Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1) #<--TODO momentum is not supported at the moment

   
alices_model = copy.deepcopy(model)
bobs_model = copy.deepcopy(model)
charlies_model = copy.deepcopy(model)

alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.01, momentum=0.9)
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.01, momentum=0.9)
charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.01, momentum=0.9)

alices_model_prev = copy.deepcopy(model)
bobs_model_prev = copy.deepcopy(model)
charlies_model_prev = copy.deepcopy(model)

#<--Train Neural network and validate with test set after completion of training every epoch


for epoch in range(1, args.epochs+1):
    

    model_a = train_non_iid(args, alices_model, device, alice_train_loader, alices_opt, epoch,0)
    model_b = train_non_iid(args, bobs_model, device, bob_train_loader, bobs_opt, epoch,1)
    model_c = train_non_iid(args, charlies_model, device, charlie_train_loader, charlies_opt, epoch,2)


    combined_model = combine(model_a,model_b,model_c,2/11,3/11,6/11)
    #combined_model = combine(model_a,model_b,model_c,1/3,1/3,1/3)

    parmod_a = copy.deepcopy(alices_model_prev)
    parmod_b = copy.deepcopy(bobs_model_prev)
    parmod_c = copy.deepcopy(charlies_model_prev)

    # alices_model = combine(parmod_a,model_a,combined_model,1,2/3,1/3)
    # bobs_model = combine(parmod_b,model_b,combined_model,1,1/2,1/2)
    # charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

    alices_model = combine(parmod_a,model_a,combined_model,1,0,0)
    bobs_model = combine(parmod_b,model_b,combined_model,1,0,0)
    charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.01, momentum=0.9)
    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.01, momentum=0.9)
    charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.01, momentum=0.9)

    alices_model_prev = copy.deepcopy(alices_model)
    bobs_model_prev = copy.deepcopy(bobs_model)
    charlies_model_prev = copy.deepcopy(charlies_model)
    
    test(args, charlies_model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

Test set: Average loss: 0.0020, Accuracy: 2921/10000 (29%)


Test set: Average loss: 0.0019, Accuracy: 3856/10000 (39%)


Test set: Average loss: 0.0024, Accuracy: 4250/10000 (42%)


Test set: Average loss: 0.0028, Accuracy: 4505/10000 (45%)


Test set: Average loss: 0.0029, Accuracy: 4538/10000 (45%)


Test set: Average loss: 0.0028, Accuracy: 4714/10000 (47%)


Test set: Average loss: 0.0028, Accuracy: 4830/10000 (48%)


Test set: Average loss: 0.0028, Accuracy: 4856/10000 (49%)


Test set: Average loss: 0.0028, Accuracy: 4896/10000 (49%)


Test set: Average loss: 0.0029, Accuracy: 4939/10000 (49%)


Test set: Average loss: 0.0027, Accuracy: 5060/10000 (51%)


Test set: Average loss: 0.0027, Accuracy: 5104/10000 (51%)


Test set: Average loss: 0.0028, Accuracy: 5068/10000 (51%)


Test set: Average loss: 0.0028, Accuracy: 5100/10000 (51%)


Test set: Average loss: 0.0026,

In [28]:
#Different size of dataset, different accuracy fractional 

#<--Load federated training data and test data
#federated_train_loader,test_loader=load_data()

##Modified

alice_train_loader,bob_train_loader,charlie_train_loader,test_loader=load_data_non_iid()

# remote_dataset = (list(), list(), list())
# train_distributed_dataset = []

# for batch_idx, (data,target) in enumerate(alice_train_loader):
#       data = data.send(alice)
#       target = target.send(alice)
#       remote_dataset[0].append((data, target))
# for batch_idx, (data,target) in enumerate(bob_train_loader):
#       data = data.send(bob)
#       target = target.send(bob)
#       remote_dataset[1].append((data, target))
# for batch_idx, (data,target) in enumerate(charlie_train_loader):
#       data = data.send(charlie)
#       target = target.send(charlie)
#       remote_dataset[2].append((data, target))

#<--Create Neural Network model instance

result_matrix = []

#<--Train Neural network and validate with test set after completion of training every epoch

for j in range(9):

    k = j/4

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.1) #<--TODO momentum is not supported at the moment
      
    alices_model = copy.deepcopy(model)
    bobs_model = copy.deepcopy(model)
    charlies_model = copy.deepcopy(model)

    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001, momentum=0.9)
    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001, momentum=0.9)
    charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.001, momentum=0.9)

    alices_model_prev = copy.deepcopy(model)
    bobs_model_prev = copy.deepcopy(model)
    charlies_model_prev = copy.deepcopy(model)

    for epoch in range(1, args.epochs + 1):
        

        model_a = train_non_iid(args, alices_model, device, alice_train_loader, alices_opt, epoch,0)
        model_b = train_non_iid(args, bobs_model, device, bob_train_loader, bobs_opt, epoch,1)
        model_c = train_non_iid(args, charlies_model, device, charlie_train_loader, charlies_opt, epoch,2)


        combined_model = combine(model_a,model_b,model_c,2/11,3/11,6/11)
        
        parmod_a = copy.deepcopy(alices_model_prev)
        parmod_b = copy.deepcopy(bobs_model_prev)
        parmod_c = copy.deepcopy(charlies_model_prev)

        # for key in sda:
        #   sda[key] = sda[key] + (2*grad_a[key] + grad_combine[key])/3

        # for key in sdb:
        #   sdb[key] = sdb[key] + (grad_b[key] + grad_combine[key])/2
          
        # for key in sdc:
        #   sdc[key] = sdc[key] + grad_combine[key]

        alices_model = combine(parmod_a,model_a,combined_model,1,1 - pow(2/3,k),pow(1/3,k))
        bobs_model = combine(parmod_b,model_b,combined_model,1,1 - pow(1/2,k),pow(1/2,k))
        charlies_model = combine(parmod_c,model_c,combined_model,1,0,1)

        alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.001, momentum=0.9)
        bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.001, momentum=0.9)
        charlies_opt = optim.SGD(params=charlies_model.parameters(),lr=0.001, momentum=0.9)

        alices_model_prev = copy.deepcopy(alices_model)
        bobs_model_prev = copy.deepcopy(bobs_model)
        charlies_model_prev = copy.deepcopy(charlies_model)
    
    tup = []
    a = test(args, alices_model, device, test_loader)
    b = test(args, bobs_model, device, test_loader)
    c = test(args, charlies_model, device, test_loader)
    tup.append(a)
    tup.append(b)
    tup.append(c)
    result_matrix.append(tup)
    

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

Test set: Average loss: 0.0012, Accuracy: 5784/10000 (58%)


Test set: Average loss: 0.0012, Accuracy: 5784/10000 (58%)


Test set: Average loss: 0.0012, Accuracy: 5784/10000 (58%)


Test set: Average loss: 0.0013, Accuracy: 5456/10000 (55%)


Test set: Average loss: 0.0012, Accuracy: 5603/10000 (56%)


Test set: Average loss: 0.0012, Accuracy: 5603/10000 (56%)


Test set: Average loss: 0.0014, Accuracy: 5124/10000 (51%)


Test set: Average loss: 0.0013, Accuracy: 5506/10000 (55%)


Test set: Average loss: 0.0013, Accuracy: 5506/10000 (55%)


Test set: Average loss: 0.0014, Accuracy: 4916/10000 (49%)


Test set: Average loss: 0.0013, Accuracy: 5437/10000 (54%)


Test set: Average loss: 0.0013, Accuracy: 5437/10000 (54%)


Test set: Average loss: 0.0014, Accuracy: 4841/10000 (48%)


Test set: Average loss: 0.0012, Accuracy: 5613/10000 (56%)


Test set: Average loss: 0.0012,

In [20]:
test(args, alices_model, device, test_loader)



Test set: Average loss: 0.0014, Accuracy: 5165/10000 (52%)



In [55]:
for name1, param1 in model.named_parameters():
  print(name1,param1)

conv1.weight Parameter containing:
tensor([[[[ 8.0471e-02,  9.4124e-02,  9.7986e-02, -6.7676e-02,  3.9696e-02],
          [ 5.9455e-02, -6.2472e-02,  3.8949e-02, -7.7579e-02,  8.6004e-02],
          [ 9.6893e-02, -2.3372e-04, -6.9689e-02,  1.1327e-01,  1.3051e-02],
          [ 4.9144e-02,  1.1280e-02,  3.2167e-02,  3.1443e-02,  6.2604e-02],
          [-3.6947e-02, -3.1457e-02,  8.5567e-02,  1.4600e-02,  2.4051e-02]],

         [[ 7.6321e-02,  8.7397e-02, -5.8722e-02,  9.6941e-02,  8.2376e-02],
          [-8.7067e-02,  4.2338e-02,  4.6218e-02, -8.3529e-02,  4.9226e-02],
          [ 2.5465e-03,  1.0428e-01,  3.8135e-02,  1.4325e-02, -1.0602e-01],
          [ 9.1805e-02,  7.6619e-03,  2.5026e-02, -6.0241e-02, -1.1025e-01],
          [ 2.8448e-02, -1.1458e-01,  2.8051e-02, -5.8262e-02, -3.4229e-02]],

         [[ 6.4269e-02,  5.6857e-02,  4.9971e-02, -9.6964e-02,  4.6509e-02],
          [-6.8977e-02, -1.0247e-01, -9.3396e-02,  9.7901e-02, -1.1474e-01],
          [ 1.0388e-01, -1.0870e-01, 